<a href="https://colab.research.google.com/github/golecalicja/thats-what-she-said/blob/in%C5%BC/emotion_text_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import pandas as pd

file_path = '/content/drive/My Drive/Colab Notebooks/The Office/df_sentiment.csv'
df = pd.read_csv(file_path, index_col=0)
df

,season,episode,scene,line,character,line_preprocessed,sentiment,sentiment_category
0,1,1,1,All right Jim. Your quarterlies look very good...,Michael,all right jim your quarterlies look very good...,0.597857,NaN
1,1,1,1,"Oh, I told you. I couldn't close it. So...",Jim,oh i told you i could not close it so,0.000000,NaN
2,1,1,1,So you've come to the master for guidance? Is ...,Michael,so you have come to the master for guidance i...,0.000000,NaN
3,1,1,1,"Actually, you called me in here, but yeah.",Jim,actually you called me in here but yeah,0.000000,NaN
4,1,1,1,"All right. Well, let me show you how it's done.",Michael,all right well let me show you how it is done,0.285714,NaN
...,...,...,...,...,...,...,...,...
57578,9,23,112,It all seems so very arbitrary. I applied for ...,Creed,it all seems so very arbitrary i applied for ...,-0.057500,NaN
57579,9,23,113,I just feel lucky that I got a chance to share...,Meredith,i just feel lucky that i got a chance to share...,0.166667,NaN
57580,9,23,114,I'm happy that this was all filmed so I can re...,Phyllis,i am happy that this was all filmed so i can r...,0.322222,NaN
57581,9,23,115,I sold paper at this company for 12 years. My ...,Jim,i sold paper at this company for years my ...,0.060000,NaN


In [ ]:
df['line_preprocessed'] = df['line_preprocessed'].apply(lambda x: str(x))
df['line'] = df['line'].apply(lambda x: str(x))

In [ ]:
!pip install transformers

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline
import torch

tokenizer = AutoTokenizer.from_pretrained("michellejieli/emotion_text_classifier")
model = AutoModelForSequenceClassification.from_pretrained("michellejieli/emotion_text_classifier")

emotion_classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)

In [ ]:
from tqdm import tqdm
results = []
batch_size = 10

batch_size = 10
for line in tqdm(df['line']):
    try:
        result = emotion_classifier(line)
        results.append(result[0]['label'])
    except Exception as e:
        print(f"Błąd przy przetwarzaniu linii: {line}")
        print(str(e))
        results.append("Error")


df['predicted_emotion'] = results[:len(df)]

100%|██████████| 56359/56359 [1:06:21<00:00, 14.15it/s]


In [ ]:
df

,season,episode,scene,line,character,line_preprocessed,sentiment,sentiment_category,predicted_emotion
0,1,1,1,All right Jim. Your quarterlies look very good...,Michael,all right jim your quarterlies look very good...,0.597857,NaN,neutral
1,1,1,1,"Oh, I told you. I couldn't close it. So...",Jim,oh i told you i could not close it so,0.000000,NaN,neutral
2,1,1,1,So you've come to the master for guidance? Is ...,Michael,so you have come to the master for guidance i...,0.000000,NaN,neutral
3,1,1,1,"Actually, you called me in here, but yeah.",Jim,actually you called me in here but yeah,0.000000,NaN,neutral
4,1,1,1,"All right. Well, let me show you how it's done.",Michael,all right well let me show you how it is done,0.285714,NaN,neutral
...,...,...,...,...,...,...,...,...,...
57578,9,23,112,It all seems so very arbitrary. I applied for ...,Creed,it all seems so very arbitrary i applied for ...,-0.057500,NaN,neutral
57579,9,23,113,I just feel lucky that I got a chance to share...,Meredith,i just feel lucky that i got a chance to share...,0.166667,NaN,joy
57580,9,23,114,I'm happy that this was all filmed so I can re...,Phyllis,i am happy that this was all filmed so i can r...,0.322222,NaN,joy
57581,9,23,115,I sold paper at this company for 12 years. My ...,Jim,i sold paper at this company for years my ...,0.060000,NaN,joy


In [ ]:
df.to_csv('df_emotions.csv')

In [ ]:
from google.colab import files
files.download('df_emotions.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df['predicted_emotion'].value_counts()

neutral     37760
joy          5309
anger        4169
sadness      3810
fear         3258
surprise     1933
disgust       492
Name: predicted_emotion, dtype: int64

In [ ]:
df[df['predicted_emotion'] == 'neutral'].line_preprocessed

0        all right jim  your quarterlies look very good...
1              oh  i told you  i could not close it  so   
2        so you have come to the master for guidance  i...
3               actually  you called me in here  but yeah 
4         all right  well  let me show you how it is done 
                               ...                        
57958             no  but i still have my medal from that 
57960                                 why do you say that 
57963                          no  but maybe the reason   
57964                                    you   re not gay 
57965     and all the faces that i know have that same ...
Name: line_preprocessed, Length: 37760, dtype: object